In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math
import json
from sklearn.preprocessing import MinMaxScaler

In [2]:
def card_data_generator():
    '''Generates Card Data as a list of dictionaries (one dictionary for each card).'''
    
    card1 = '{"Card_ID": 1, "Card_Name" : "Cash Wise", "Credit_Score": 645, "Card_Desc": "Cash Wise Card", "Intro" : [{"Intro_ID" : 1, "Category" : "Any", "Threshold" : 1000, "Reward" : 1500}]}'
    card2 = '{"Card_ID": 2, "Card_Name" : "Platinum", "Credit_Score": 600, "Card_Desc": "Platinum Card"}'
    card3 = '{"Card_ID": 3, "Card_Name" : "Hotel", "Credit_Score": 635, "Card_Desc": "Hotel Card", "Intro" : [{"Intro_ID" : 1, "Category" : "Hotel", "Threshold" : 1000, "Reward" : 1500}]}'
    card4 = '{"Card_ID": 4, "Card_Name" : "College", "Credit_Score": 0, "Card_Desc": "College Card", "Intro" : [{"Intro_ID" : 1, "Category" : "Education", "Threshold" : 800, "Reward" : 1000}]}'
    card5 = '{"Card_ID": 5, "Card_Name" : "Visa Signature", "Credit_Score": 680, "Card_Desc": "Visa Signature Card"}'
    card6 = '{"Card_ID": 6, "Card_Name" : "Holiday", "Credit_Score": 618, "Card_Desc": "Holiday Card", "Intro" : [{"Intro_ID" : 1, "Category" : "Travel", "Threshold" : 1000, "Reward" : 500}]}'
    card7 = '{"Card_ID": 7, "Card_Name" : "Shopping", "Credit_Score": 660, "Card_Desc": "Shopping Card", "Intro" : [{"Intro_ID" : 1, "Category" : "Shop_pos", "Threshold" : 500, "Reward" : 1500},\
                                                                                                                    {"Intro_ID" : 2, "Category" : "Shop_net", "Threshold" : 500, "Reward" : 1000}]}'
    card8 = '{"Card_ID": 8, "Card_Name" : "Entertainment", "Credit_Score": 630, "Card_Desc": "Entertainment Card", "Intro" : [{"Intro_ID" : 1, "Category" : "Entertainment", "Threshold" : 1000, "Reward" : 1500}]}'
    card9 = '{"Card_ID": 9, "Card_Name" : "Credit Builder", "Credit_Score": 0, "Card_Desc": "Credit Builder Card"}'
    card1 = json.loads(card1)
    card2 = json.loads(card2)
    card3 = json.loads(card3)
    card4 = json.loads(card4)
    card5 = json.loads(card5)
    card6 = json.loads(card6)
    card7 = json.loads(card7)
    card8 = json.loads(card8)
    card9 = json.loads(card9)
    card_data = []
    card_data.append(card1)
    card_data.append(card2)
    card_data.append(card3)
    card_data.append(card4)
    card_data.append(card5)
    card_data.append(card6)
    card_data.append(card7)
    card_data.append(card8)
    card_data.append(card9)
    return card_data

In [3]:
def category_card_mapper():
    '''Generates mapping from Card Name to Card Number (Index)'''
    
    cat_map = pd.read_excel("Cat_Map.xlsx")
    cat_map.sort_values(by = 'Category_Name', inplace = True,ignore_index = True)
    indices = pd.Series(cat_map.index, index = cat_map['Category_Name'])
    return cat_map, indices

In [4]:
def card_map_generator(card_data):
    card_mapper = {}
    for i, card in enumerate(card_data):
        card_mapper[card['Card_Name']] = i
    return card_mapper

In [5]:
def trans_wise_rewards(df, card_data, card_mapper,indices,cat_map):

    rewards = [] ## Transaction wise rewards
    check = {} ## To check if intro benefit has been given or not on a particular category(it can be in multiple categories)
    check_any = {} ## To check if intro benefit has been given if it was in any category
    cat_wise = {} ## Category wise reward points for different users
    cat_wise_amt = {} ## Category wise amount spent for different users

    for i in range(0,df.shape[0]): ## The transactions
        amount = df.loc[i,"trans_amt"].astype(int) ## amount spent
        card_name = df.loc[i,"card_type"] ## Card used
        card = card_data[card_mapper[card_name]] ## Details of the card used
        cat = indices[df.loc[i,"purchase_category"]] ## Purchase Category
        card_percent = cat_map[card_name][cat] ## Percent rewards in that category for the card used
        card_percent = card_percent/10
        ans = card_percent*amount ## Rewards

        card_no = df.loc[i,"card_num"] ## Card Number(here it also works as user number)
        if card_no in cat_wise.keys(): ## if card number already present in dictionary
            cat_wise[card_no][cat] += ans ## add rewards to the dictionary corresponding to the purchase category
            cat_wise_amt[card_no][cat] += amount ## amount spent in the category

        else: ## If card number not present in the dictionary add it
            cat_wise[card_no] = [0 for j in range(0,16)] ## 16 Categories initialized to 0
            cat_wise_amt[card_no] = [0 for j in range(0,16)] ## 16 Categories initialized to 0
            check[card_no] = [0 for j in range(0,16)] ## Intro benefit not given yet on any of the categories
            check_any[card_no] = 0
            cat_wise[card_no][cat] += ans ## add rewards to the purchase category
            cat_wise_amt[card_no][cat] += amount  ## amount spent in the category

        if 'Intro' in card.keys():
            for intro in card['Intro']: ## Intro benefits of the card used
                category = intro['Category'] ## Intro benefit on which category
                threshold = intro['Threshold'] ## The amount threshold
                reward = intro['Reward'] ## The Intro reward

                if category == 'Any': ## If reward is given on just the amount spent not a particular category
                    if sum(cat_wise_amt[card_no]) > threshold and check_any[card_no] == 0:
                        ans = ans + reward
                        cat_wise[card_no][cat] += reward
                        check_any[card_no] = 1 ## Intro reward given
                elif cat_wise_amt[card_no][indices[category]] >= threshold and check[card_no][indices[category]] == 0: ## If reward is on a particular category
                    ans = ans + reward
                    cat_wise[card_no][cat] += reward 
                    check[card_no][indices[category]] = 1 ## Intro reward given
        rewards.append(ans)

    return rewards, cat_wise

In [6]:
def cat_wise_info(cat_wise):
    cat_wise_mean = []
    cat_wise_std = []
    for j in range(0,16):
        arr = []
        for i in cat_wise:
            arr.append(cat_wise[i][j])
        cat_wise_std.append(np.std(arr))
        cat_wise_mean.append(sum(arr)/len(cat_wise))
    return cat_wise_std,cat_wise_mean

In [7]:
def user_wise_rewards(cat_wise):
    user_wise = []
    for i in cat_wise:
        user_wise.append(sum(cat_wise[i]))
    return user_wise

In [16]:
def main():
    df = pd.read_csv("Final Transaction Data.csv") ## Transaction data
    card_data = card_data_generator() ## Card Data
    cat_map, indices = category_card_mapper() ## Category wise mapping of cards
    card_mapper = card_map_generator(card_data) ## card and it's id
    rewards, cat_wise = trans_wise_rewards(df, card_data, card_mapper,indices,cat_map) ##Transaction wise rewards
    
    
    cat_wise_std,cat_wise_mean = cat_wise_info(cat_wise)
    user_wise = user_wise_rewards(cat_wise)
    udf = pd.read_csv("Updated User Database.csv")
    udf['rewards'] = user_wise
    udf.to_csv('Updated User Rewards', index = False)
    user_df = pd.read_csv("Supervised User Database.csv")
    user_df['rewards'] = user_wise
    user_df.to_csv('Superwised database with rewards.csv', index = False)
    print(user_df)

In [17]:
main()

     User_Id            job  credit_score card_issue_date  card_type  \
0          0         doctor           755      2019-01-01          4   
1          1  self employed           744      2019-01-01          4   
2          2  self employed           629      2019-01-01          8   
3          3       engineer           634      2019-01-01          5   
4          4         artist           765      2019-01-01          4   
..       ...            ...           ...             ...        ...   
994      994     politician           639      2020-10-24          7   
995      995        student           627      2020-10-25          1   
996      996     accountant           629      2020-10-31          1   
997      997  self employed           774      2020-12-02          4   
998      998       engineer           850      2020-12-21          6   

     Education  Entertainment     Food  Gas_trans  Grocery_net  ...    Hotel  \
0          0.0        19455.0  11672.0     5875.0      

In [19]:
df = pd.read_csv("Updated User Rewards")

In [20]:
df

,User_Id,job,credit_score,card_issue_date,card_type,Education,Entertainment,Food,Gas_trans,Grocery_net,...,Home,Hotel,Kids_pets,Misc_net,Misc_pos,Personal,Shop_net,Shop_pos,Travel,rewards
0,0,doctor,755,2019-01-01,4,0.0,19455.0,11672.0,5875.0,12482.0,...,19380.0,23277.0,15328.0,1194.0,1741.0,12465.0,33533.0,34416.0,1799.0,150059.3
1,1,self employed,744,2019-01-01,4,0.0,12655.0,6777.0,36155.0,6817.0,...,20888.0,23216.0,16622.0,2682.0,6248.0,15719.0,18038.0,25880.0,1937.0,139866.1
2,2,self employed,629,2019-01-01,8,0.0,2678.0,4129.0,4913.0,1844.0,...,3392.0,3569.0,4972.0,311.0,269.0,2660.0,5698.0,5095.0,1404.0,3828.0
3,3,engineer,634,2019-01-01,5,0.0,2564.0,2177.0,5618.0,240.0,...,3535.0,4195.0,2852.0,1430.0,354.0,3585.0,8195.0,2290.0,8324.0,13604.7
4,4,artist,765,2019-01-01,4,0.0,18875.0,15472.0,8799.0,7030.0,...,26298.0,7153.0,12862.0,0.0,7688.0,7359.0,25877.0,31034.0,54547.0,155572.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,994,politician,639,2020-10-24,7,0.0,0.0,0.0,13.0,0.0,...,0.0,832.0,0.0,0.0,0.0,0.0,3864.0,924.0,0.0,1441.6
995,995,student,627,2020-10-25,1,614.0,0.0,0.0,12.0,0.0,...,0.0,0.0,0.0,772.0,0.0,0.0,1834.0,1038.0,0.0,427.0
996,996,accountant,629,2020-10-31,1,0.0,0.0,0.0,8.0,0.0,...,0.0,785.0,23.0,0.0,8.0,0.0,3020.0,0.0,10.0,417.1
997,997,self employed,774,2020-12-02,4,745.0,0.0,0.0,9.0,0.0,...,0.0,0.0,0.0,781.0,0.0,0.0,5850.0,3619.0,0.0,6651.2


In [21]:
df = pd.read_csv("Superwised database with rewards.csv")

In [22]:
df

,User_Id,job,credit_score,card_issue_date,card_type,Education,Entertainment,Food,Gas_trans,Grocery_net,...,Hotel,Kids_pets,Misc_net,Misc_pos,Personal,Shop_net,Shop_pos,Travel,best_card,rewards
0,0,doctor,755,2019-01-01,4,0.0,19455.0,11672.0,5875.0,12482.0,...,23277.0,15328.0,1194.0,1741.0,12465.0,33533.0,34416.0,1799.0,4,150059.3
1,1,self employed,744,2019-01-01,4,0.0,12655.0,6777.0,36155.0,6817.0,...,23216.0,16622.0,2682.0,6248.0,15719.0,18038.0,25880.0,1937.0,4,139866.1
2,2,self employed,629,2019-01-01,8,0.0,2678.0,4129.0,4913.0,1844.0,...,3569.0,4972.0,311.0,269.0,2660.0,5698.0,5095.0,1404.0,2,3828.0
3,3,engineer,634,2019-01-01,5,0.0,2564.0,2177.0,5618.0,240.0,...,4195.0,2852.0,1430.0,354.0,3585.0,8195.0,2290.0,8324.0,2,13604.7
4,4,artist,765,2019-01-01,4,0.0,18875.0,15472.0,8799.0,7030.0,...,7153.0,12862.0,0.0,7688.0,7359.0,25877.0,31034.0,54547.0,4,155572.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,994,politician,639,2020-10-24,7,0.0,0.0,0.0,13.0,0.0,...,832.0,0.0,0.0,0.0,0.0,3864.0,924.0,0.0,1,1441.6
995,995,student,627,2020-10-25,1,614.0,0.0,0.0,12.0,0.0,...,0.0,0.0,772.0,0.0,0.0,1834.0,1038.0,0.0,1,427.0
996,996,accountant,629,2020-10-31,1,0.0,0.0,0.0,8.0,0.0,...,785.0,23.0,0.0,8.0,0.0,3020.0,0.0,10.0,1,417.1
997,997,self employed,774,2020-12-02,4,745.0,0.0,0.0,9.0,0.0,...,0.0,0.0,781.0,0.0,0.0,5850.0,3619.0,0.0,6,6651.2
